In [302]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [303]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

workers - workers.csv


,worker_id,name,плоская,высокая,струйная
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


equipment - equipment.csv


,machine_id,machine_type,operators_needed
0,PM-01,плоская,4
1,PM-02,плоская,4
2,PM-03,высокая,4
3,PM-04,высокая,4
4,SM-01,струйная,1


schedule - schedule_template.csv


,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


requirements - position_requirements.csv


,machine_type,position,min_rank,profession_required
0,плоская,1,7,Печатник
1,плоская,2,6,Печатник
2,плоская,3,5,Печатник
3,плоская,4,4,Печатник
4,струйная,1,5,Оператор струйной печати


plan - plan.csv


,machine_id,night,day,evening
0,PM-01,True,True,True
1,PM-02,True,True,True
2,PM-03,True,True,True
3,PM-04,True,True,True
4,SM-01,True,True,True


In [304]:
# Определяем основную профессию
cols = ["плоская", "высокая", "струйная"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,W001,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,W002,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,W003,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,W004,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,W005,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [305]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
target_shift = "День"
shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}

In [306]:
# Функция для переворота смены
def shift_next_week(week, shift_name):
    shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}
    print(f"shift_workers - Кандидаты для работы в смену: {shift_name} недели {week}")
    result = schedule[
        (schedule["week"] == week - 1)
        &
        # (schedule["day"] == target_day) &
        (schedule["shift"] == shift[shift_name])
    ]
    result = result.merge(workers, on="worker_id", how="left")
    print(f"В смене работает {len(result)} человек")

    return result

In [307]:
shift_workers_day = shift_next_week(target_week, "День")
display(shift_workers_day.head())

shift_workers_evning = shift_next_week(target_week, "Вечер")
# display(shift_workers_evning)

shift_workers_nigth = shift_next_week(target_week, "Ночь")
# display(shift_workers_nigth)

shift_workers - Кандидаты для работы в смену: День недели 2
В смене работает 20 человек


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,1,Вечер,W021,Работник П 21,7,6,0,плоская,"[плоская, высокая]"
1,1,Вечер,W022,Работник П 22,6,5,0,плоская,"[плоская, высокая]"
2,1,Вечер,W023,Работник П 23,5,4,0,плоская,"[плоская, высокая]"
3,1,Вечер,W024,Работник П 24,4,0,5,струйная,"[плоская, струйная]"
4,1,Вечер,W025,Работник П 25,7,6,0,плоская,"[плоская, высокая]"


shift_workers - Кандидаты для работы в смену: Вечер недели 2
В смене работает 20 человек
shift_workers - Кандидаты для работы в смену: Ночь недели 2
В смене работает 20 человек


In [308]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None
    
    return shift_equipment

In [324]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# Заменяем названия смен на русские
plan_long = plan_long.replace(
    {"shift": {"night": "Ночь", "day": "День", "evening": "Вечер"}}
)

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week

# Позже убрать дублирование кода
shift_equipment = f_shift_equipment(plan_long, target_shift)

shift_equipment_day = f_shift_equipment(plan_long, 'День')
shift_equipment_night = f_shift_equipment(plan_long, 'Вечер')
shift_equipment_evening = f_shift_equipment(plan_long, 'Ночь')

display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,None
1,2,День,PM-01,плоская,2,6,Печатник,None
2,2,День,PM-01,плоская,3,5,Печатник,None
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,None
1,2,Вечер,PM-01,плоская,2,6,Печатник,None
2,2,Вечер,PM-01,плоская,3,5,Печатник,None
3,2,Вечер,PM-01,плоская,4,4,Печатник,None
4,2,Вечер,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None


In [325]:
def candidat(s, profesion, min_rank):

    if s == 'ferst':
        candidates = shift_workers_day[
        (shift_workers_day["основная_профессия"] == profesion)
        & (shift_workers_day[profesion] == min_rank)
        & (~shift_workers_day["worker_id"].isin(appointed))
            ]
    elif s == 'second':
        candidates = shift_workers_day[
        (shift_workers_day["все_профессии"].apply(lambda x: profesion in x))
        & (shift_workers_day[profesion] >= min_rank)
        & (~shift_workers_day["worker_id"].isin(appointed))
            ]
    return candidates



def position_assignment(shift_equipment, s='ferst'):
    # appointed = set()
    free_machin_positions = []
    
    for index, row in shift_equipment.iterrows():
        if not row["worker_id"]:
            print(row["worker_id"], "->", end=" ")
            profesion = row["machine_type"]
            min_rank = row["min_rank"]

            candidates = candidat(s, profesion, min_rank)

            # candidates = shift_workers_day[
            #     (shift_workers_day["основная_профессия"] == profesion)
            #     & (shift_workers_day[profesion] == min_rank)
            #     & (~shift_workers_day["worker_id"].isin(appointed))
            # ]
            # print(candidates)

            if not candidates.empty:
                first = candidates.iloc[0]  # <-- первая строка
                worker_id = first["worker_id"]

                shift_equipment.loc[index, "worker_id"] = worker_id  # <-- loc, не iloc
                appointed.add(worker_id)
                print(worker_id)
            else:
                # при необходимости логировать отсутствие кандидатов
                print(f"Нет кандидатов для index={index}, {profesion}, min_rank={min_rank}")
                free_machin_positions.append(shift_equipment.loc[index])
        else:
            print(row["worker_id"], "уже назначен")
        
    return pd.DataFrame(free_machin_positions), shift_equipment

appointed = set()

In [327]:
free_machin_positions, shift_equipment_day = position_assignment(shift_equipment_day, 'ferst')

# Выводим свободные позиции
# free_machin_positions = pd.DataFrame(free_machin_positions)
print("Свободные позиции оборудования:")
display(free_machin_positions)

free_shift_workers = shift_workers_day[~shift_workers_day["worker_id"].isin(appointed)]
print("Свободные кандидаты для работы в смену: День")
display(free_shift_workers)


if not free_machin_positions.empty:
    free_machin_positions, shift_equ = position_assignment(free_machin_positions, 'second')
    shift_equipment_day = shift_equipment_day.combine_first(shift_equ)


display(shift_equipment_day)

W021 уже назначен
W022 уже назначен
W023 уже назначен
W037 уже назначен
W025 уже назначен
W026 уже назначен
W027 уже назначен
W038 уже назначен
W029 уже назначен
W030 уже назначен
W031 уже назначен
W039 уже назначен
W033 уже назначен
W034 уже назначен
W035 уже назначен
W040 уже назначен
W024 уже назначен
W028 уже назначен
W032 уже назначен
W036 уже назначен
Свободные позиции оборудования:


""


Свободные кандидаты для работы в смену: День


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,W021
1,2,День,PM-01,плоская,2,6,Печатник,W022
2,2,День,PM-01,плоская,3,5,Печатник,W023
3,2,День,PM-01,плоская,4,4,Печатник,W037
4,2,День,PM-02,плоская,1,7,Печатник,W025
5,2,День,PM-02,плоская,2,6,Печатник,W026
6,2,День,PM-02,плоская,3,5,Печатник,W027
7,2,День,PM-02,плоская,4,4,Печатник,W038
8,2,День,PM-03,высокая,1,7,Печатник,W029
9,2,День,PM-03,высокая,2,6,Печатник,W030


In [322]:
# Выводим свободные позиции
# free_machin_positions = pd.DataFrame(free_machin_positions)
print("Свободные позиции оборудования:")
display(free_machin_positions)

free_shift_workers = shift_workers_day[~shift_workers_day["worker_id"].isin(appointed)]
print("Свободные кандидаты для работы в смену: День")
display(free_shift_workers)

Свободные позиции оборудования:


""


Свободные кандидаты для работы в смену: День


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


In [313]:
shift_equipment_day


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,W021
1,2,День,PM-01,плоская,2,6,Печатник,W022
2,2,День,PM-01,плоская,3,5,Печатник,W023
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,W025
5,2,День,PM-02,плоская,2,6,Печатник,W026
6,2,День,PM-02,плоская,3,5,Печатник,W027
7,2,День,PM-02,плоская,4,4,Печатник,None
8,2,День,PM-03,высокая,1,7,Печатник,W029
9,2,День,PM-03,высокая,2,6,Печатник,W030


In [328]:
print(
    f"Назначено позиций: {shift_equipment_day['worker_id'].notna().sum()} из {len(shift_equipment_day)}"
)
if not free_machin_positions.empty:
    print("Не удалось назначить (после двух проходов):")
    display(
        free_machin_positions[["machine_id", "position", "machine_type", "min_rank"]]
    )

print(len(appointed))

Назначено позиций: 20 из 20
20


In [330]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_day[~shift_workers_day["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


In [332]:
# Выводим результат с именами работников
result = shift_equipment_day.merge(
    workers[["worker_id", "name"]], on="worker_id", how="left"
)
result

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id,name
0,2,День,PM-01,плоская,1,7,Печатник,W021,Работник П 21
1,2,День,PM-01,плоская,2,6,Печатник,W022,Работник П 22
2,2,День,PM-01,плоская,3,5,Печатник,W023,Работник П 23
3,2,День,PM-01,плоская,4,4,Печатник,W037,Работник О 37
4,2,День,PM-02,плоская,1,7,Печатник,W025,Работник П 25
5,2,День,PM-02,плоская,2,6,Печатник,W026,Работник П 26
6,2,День,PM-02,плоская,3,5,Печатник,W027,Работник П 27
7,2,День,PM-02,плоская,4,4,Печатник,W038,Работник О 38
8,2,День,PM-03,высокая,1,7,Печатник,W029,Работник В 29
9,2,День,PM-03,высокая,2,6,Печатник,W030,Работник В 30


In [333]:
# Группируем по машинам и позициям, чтобы получить минимальный разряд и требуемые профессии
by_machine_position = result.groupby(
    ["week", "shift", "machine_type", "machine_id", "position"], as_index=True
).agg(
    # machine_type=('machine_type', 'first'),
    # position = ('position', 'first'),
    min_rank=("min_rank", "min"),
    worker_id=("worker_id", "first"),
    name=("name", "first"),
    profession_required=("profession_required", "first"),
)

display(by_machine_position)

min_rank worker_id  \
week shift machine_type machine_id position                       
2    День  высокая      PM-03      1                7      W029   
                                   2                6      W030   
                                   3                5      W031   
                                   4                4      W039   
                        PM-04      1                7      W033   
                                   2                6      W034   
                                   3                5      W035   
                                   4                4      W040   
           плоская      PM-01      1                7      W021   
                                   2                6      W022   
                                   3                5      W023   
                                   4                4      W037   
                        PM-02      1                7      W025   
                                   2                6      W026   
                                   3                5      W027   
                                   4                4      W038   
           струйная     SM-01      1                5      W024   
                        SM-02      1                5      W028   
                        SM-03      1                5      W032   
                        SM-04      1                5      W036   

                                                      name  \
week shift machine_type machine_id position                  
2    День  высокая      PM-03      1         Работник В 29   
                                   2         Работник В 30   
                                   3         Работник В 31   
                                   4         Работник О 39   
                        PM-04      1         Работник В 33   
                                   2         Работник В 34   
                                   3         Работник В 35   
                                   4         Работник О 40   
           плоская      PM-01      1         Работник П 21   
                                   2         Работник П 22   
                                   3         Работник П 23   
                                   4         Работник О 37   
                        PM-02      1         Работник П 25   
                                   2         Работник П 26   
                                   3         Работник П 27   
                                   4         Работник О 38   
           струйная     SM-01      1         Работник П 24   
                        SM-02      1         Работник П 28   
                        SM-03      1         Работник В 32   
                        SM-04      1         Работник В 36   

                                                  profession_required  
week shift machine_type machine_id position                            
2    День  высокая      PM-03      1                         Печатник  
                                   2                         Печатник  
                                   3                         Печатник  
                                   4                         Печатник  
                        PM-04      1                         Печатник  
                                   2                         Печатник  
                                   3                         Печатник  
                                   4                         Печатник  
           плоская      PM-01      1                         Печатник  
                                   2                         Печатник  
                                   3                         Печатник  
                                   4                         Печатник  
                        PM-02      1                         Печатник  
                                   2                         Печатник  
                                   3                 

In [319]:
by_machine_position = by_machine_position.reset_index()
by_machine_position

,week,shift,machine_type,machine_id,position,min_rank,worker_id,name,profession_required
0,2,День,высокая,PM-03,1,7,None,None,Печатник
1,2,День,высокая,PM-03,2,6,None,None,Печатник
2,2,День,высокая,PM-03,3,5,None,None,Печатник
3,2,День,высокая,PM-03,4,4,None,None,Печатник
4,2,День,высокая,PM-04,1,7,None,None,Печатник
5,2,День,высокая,PM-04,2,6,None,None,Печатник
6,2,День,высокая,PM-04,3,5,None,None,Печатник
7,2,День,высокая,PM-04,4,4,None,None,Печатник
8,2,День,плоская,PM-01,1,7,None,None,Печатник
9,2,День,плоская,PM-01,2,6,None,None,Печатник


In [320]:
schedule.head()

,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


In [321]:
temp = by_machine_position[["week", "shift", "worker_id"]]
temp

,week,shift,worker_id
0,2,День,None
1,2,День,None
2,2,День,None
3,2,День,None
4,2,День,None
5,2,День,None
6,2,День,None
7,2,День,None
8,2,День,None
9,2,День,None
